In [1]:
from networks.LUTDeiT import LUT_DeiT
pl_model = LUT_DeiT().load_from_checkpoint("/home/yllab/JiaXing/Research/epoch=1-step=98.ckpt")

In [2]:
model = pl_model.model

In [ ]:
print(model)

In [1]:
import torch
# torch.save(model, "amm_vit_082.pth")
# model = torch.load("amm_vit_082.pth", map_location="cpu")
model = torch.load("amm_vit_082.pth", map_location="cpu")
input_tensors = (torch.randn(16, 3, 224, 224),)

In [2]:
scripted_model = torch.jit.trace(model, input_tensors).eval()

/home/yllab/anaconda3/envs/vlsi39/lib/python3.9/site-packages/torch/__init__.py:1404: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert condition, message
/home/yllab/JiaXing/Research/operations/amm_conv2d.py:98: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  torch.tensor(min_val).to(lut.device),
/home/yllab/JiaXing/Research/operations/amm_conv2d.py:99: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of

In [3]:
import torch
import os
from torch.utils.data import DataLoader
from torchvision import datasets
from timm import create_model
from timm.data import resolve_model_data_config, create_transform
from tqdm import tqdm
def load_data(batchSize, 
              num_workers,
              float_model
              ):
    batch_size = batchSize
    # traindir = os.path.join("/work/u1887834/imagenet/", 'train')
    valdir = os.path.join("/dataset/imagenet/", 'val')

    data_config = resolve_model_data_config(float_model)
    val_transform = create_transform(**data_config, is_training=False)
    train_transform = create_transform(**data_config, is_training=True)
    train_dataset = datasets.ImageFolder(
        valdir,
        train_transform
        )

    val_dataset = datasets.ImageFolder(
        valdir,
        val_transform
        )
   
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True,
        num_workers=num_workers, pin_memory=True, sampler=None)
    # train_loader = None
    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=True, sampler=None)
    return val_loader, val_loader
model_name = "deit3_small_patch16_224.fb_in22k_ft_in1k"
float_model = create_model(model_name, pretrained=True)
train_loader, val_loader = load_data(
        16, 
        1,
        float_model
        )
correct = 0
total = 0

# 遍歷測試數據集，並計算Top-1精確度
with torch.no_grad():
    for data in tqdm(val_loader):
        images, labels = data
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        # print(outputs.shape)
        _, predicted = torch.max(outputs, 1)
        # print(predicted) # batch one hot of 1000...
        # print(labels)
        correct += torch.eq(predicted, labels).sum().item()


top1_accuracy = (correct / len(val_loader)) * 100.0
print(f"Top-1 Accuracy: {top1_accuracy:.2f}%")
    

  1%|          | 29/3125 [00:53<1:37:23,  1.89s/it]

In [4]:
valdir = os.path.join("/dataset/imagenet/", 'val')

data_config = resolve_model_data_config(float_model)
val_transform = create_transform(**data_config, is_training=False)
val_dataset = datasets.ImageFolder(
        valdir,
        val_transform
        )
top1_accuracy = (correct / len(val_dataset)) * 100.0

In [5]:
print(f"Top-1 Accuracy: {top1_accuracy:.2f}%")

Top-1 Accuracy: 82.06%
